# Forecast of O3 data

https://www.altumintelligence.com/articles/a/Time-Series-Prediction-Using-LSTM-Deep-Neural-Networks

https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/

https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/

## A note on units

Units are not specified directly within the processed data file, `cleaned_O3_data.csv`. Units of measure are:
* `O3`: ozone, in ppb (parts per billion)
* `T`: air temperature, in degrees Fahrenheit
* `WD`: wind direction, in degrees
* `WS`: wind speed, in mph (miles per hour), average hourly

## Import the required modules

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_selector

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input
from scikeras.wrappers import KerasRegressor

2022-05-06 18:06:40.839785: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-06 18:06:40.839808: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Import the processed data

In [4]:
df = pd.read_csv("cleaned_data/cleaned_O3_data.csv")
df["DateTime"] = pd.to_datetime(df["DateTime"])
df.head()

,DateTime,Year,Month,DayOfMonth,DayOfWeek,DayOfYear,Hour,O3,T,WS,WD,ExceedsLimit
0,2000-01-01 00:00:00,2000,1,1,5,1,0,1.0,53.5,0.6,356.6,False
1,2000-01-01 01:00:00,2000,1,1,5,1,1,1.0,53.6,1.1,0.7,False
2,2000-01-01 02:00:00,2000,1,1,5,1,2,1.0,53.8,1.0,15.4,False
3,2000-01-01 03:00:00,2000,1,1,5,1,3,2.0,53.6,1.3,33.8,False
4,2000-01-01 04:00:00,2000,1,1,5,1,4,0.0,53.4,1.9,30.6,False


## Drop rows with NaN values

In [5]:
df.dropna(how = "any", inplace = True)

In [6]:
df.isna().describe()

,DateTime,Year,Month,DayOfMonth,DayOfWeek,DayOfYear,Hour,O3,T,WS,WD,ExceedsLimit
count,157137,157137,157137,157137,157137,157137,157137,157137,157137,157137,157137,157137
unique,1,1,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,False,False
freq,157137,157137,157137,157137,157137,157137,157137,157137,157137,157137,157137,157137


## Preprocessing setup: Prepare to restructure tabular data

The data must be restructured to prepare for timeseries learning. Create a function to convert the tabular data into windowed format, in which each `O3` value for prediction is paired with an array of historical feature data (including historical `O3` information) over a given window size.

In [7]:
def structure_timeseries(df = None,
                         y_col = None,
                         dt_col = None,
                         window = None,
                         use_array = True):
    
    """Prepares tabular data for timeseries learning by converting to a windowed format.
    The returned features are listed in reverse chronological order.
    
    Keyword arguments:
         df = time series features as a pandas dataframe,
         y_col = column name containing the label to be predicted,
         dt_col = column name containing the date/time stamp,
         window = number of historical feature rows to be used in
                  generating a prediction.
         use_array = boolean. Returned the processed features as a numpy array
                     (default) or as a pandas dataframe (for diagnostic purposes).    
    """
    
    
    timeseries_x = []
    timeseries_y = []    
    timeseries_dt = []

    window += 1
    
    for i in range (window, len(df)):
        # Extract a slice of the dataset, with one row larger than is required by the given window.
        # The extra row is the "current" measurement (datetime, label), and the remaining rows
        # are the "historical" data (features, labels).
        
        timestep = df[i:i - window:-1].reset_index(drop = True)
        rownames = ["t-" + str(j) for j in range(0, window)]
        timestep.index = rownames
        
        dt_stamp = timestep.iloc[0][dt_col]
        y = timestep.iloc[0][y_col]
        
        timestep = timestep.drop(dt_col, axis = "columns")
        timestep = timestep.drop("t-0", axis = 0)
        
        # Append the new windowed data to the relevant processed lists.
        # Allow for returning the features as an array or a dataframe
        # with labeled rows/columns for diagnostic purposes.
        if use_array:
            timeseries_x.append(timestep.values)
        else:
            timeseries_x.append(timestep)
        timeseries_y.append(y)
        timeseries_dt.append(dt_stamp)
    
    return timeseries_dt, timeseries_y, timeseries_x

Visually inspect the first few rows to ensure the fuction is behaving as expected:

In [8]:
df = df[["DateTime", "O3", "T", "WS", "WD"]]
df[:10]

,DateTime,O3,T,WS,WD
0,2000-01-01 00:00:00,1.0,53.5,0.6,356.6
1,2000-01-01 01:00:00,1.0,53.6,1.1,0.7
2,2000-01-01 02:00:00,1.0,53.8,1.0,15.4
3,2000-01-01 03:00:00,2.0,53.6,1.3,33.8
4,2000-01-01 04:00:00,0.0,53.4,1.9,30.6
5,2000-01-01 05:00:00,2.0,53.7,5.1,27.1
6,2000-01-01 06:00:00,1.0,53.9,2.5,37.5
7,2000-01-01 07:00:00,1.0,53.0,2.7,36.5
8,2000-01-01 08:00:00,2.0,52.0,3.4,60.6
9,2000-01-01 09:00:00,4.0,53.1,7.1,83.9


In [9]:
trial_dt, trial_y, trial_x = structure_timeseries(df = df[:10], y_col = "O3", dt_col = "DateTime", window = 5, use_array = False)
list(zip(trial_dt, trial_y, trial_x))

[(Timestamp('2000-01-01 06:00:00'),
  1.0,
        O3     T   WS    WD
  t-1  2.0  53.7  5.1  27.1
  t-2  0.0  53.4  1.9  30.6
  t-3  2.0  53.6  1.3  33.8
  t-4  1.0  53.8  1.0  15.4
  t-5  1.0  53.6  1.1   0.7),
 (Timestamp('2000-01-01 07:00:00'),
  1.0,
        O3     T   WS    WD
  t-1  1.0  53.9  2.5  37.5
  t-2  2.0  53.7  5.1  27.1
  t-3  0.0  53.4  1.9  30.6
  t-4  2.0  53.6  1.3  33.8
  t-5  1.0  53.8  1.0  15.4),
 (Timestamp('2000-01-01 08:00:00'),
  2.0,
        O3     T   WS    WD
  t-1  1.0  53.0  2.7  36.5
  t-2  1.0  53.9  2.5  37.5
  t-3  2.0  53.7  5.1  27.1
  t-4  0.0  53.4  1.9  30.6
  t-5  2.0  53.6  1.3  33.8),
 (Timestamp('2000-01-01 09:00:00'),
  4.0,
        O3     T   WS    WD
  t-1  2.0  52.0  3.4  60.6
  t-2  1.0  53.0  2.7  36.5
  t-3  1.0  53.9  2.5  37.5
  t-4  2.0  53.7  5.1  27.1
  t-5  0.0  53.4  1.9  30.6)]

Yes, the `structure_timeseries` function is preparing the data as needed. Each `O3` value is paired with a timestamp and relevant historical feature data over the specified window size. The features are given in reverse chronological order as desired.

## Preprocessing setup: Prepare to transform wind data

The prediction's inputs will include wind speed & wind direction, but prior EDA indicates that vectorized wind data (speeds in the north-south and east-west directions) is more valuable. 

Transform the wind data from speed + direction to vectors.

In [10]:
# Build a function transformer to vectorize the wind data as required.

def vectorize(wind_df):
    """Accepts wind speeds and wind directions as dataframe wind_df"""
    
    wind_df["N_wind_vec"] = np.cos(np.radians(wind_df["WD"])) * wind_df["WS"]
    wind_df["E_wind_vec"] = np.sin(np.radians(wind_df["WD"])) * wind_df["WS"]
    
    wind_df = wind_df.drop(["WS", "WD"], axis = "columns")
    return wind_df

Note that the `vectorize` function returns a new dataframe with the `WS` and `WD` columns removed; once vectorized, the original wind data is no longer needed and is dropped.

Confirm that this function is working as expected:

In [11]:
pd.concat([df[:10][["WS", "WD"]], vectorize(df[:10])], axis = 1)

/tmp/ipykernel_7171/3204449742.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_df["N_wind_vec"] = np.cos(np.radians(wind_df["WD"])) * wind_df["WS"]
/tmp/ipykernel_7171/3204449742.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_df["E_wind_vec"] = np.sin(np.radians(wind_df["WD"])) * wind_df["WS"]


,WS,WD,DateTime,O3,T,N_wind_vec,E_wind_vec
0,0.6,356.6,2000-01-01 00:00:00,1.0,53.5,0.598944,-0.035584
1,1.1,0.7,2000-01-01 01:00:00,1.0,53.6,1.099918,0.013439
2,1.0,15.4,2000-01-01 02:00:00,1.0,53.8,0.964095,0.265556
3,1.3,33.8,2000-01-01 03:00:00,2.0,53.6,1.080280,0.723184
4,1.9,30.6,2000-01-01 04:00:00,0.0,53.4,1.635410,0.967179
5,5.1,27.1,2000-01-01 05:00:00,2.0,53.7,4.540085,2.323279
6,2.5,37.5,2000-01-01 06:00:00,1.0,53.9,1.983383,1.521904
7,2.7,36.5,2000-01-01 07:00:00,1.0,53.0,2.170414,1.606022
8,3.4,60.6,2000-01-01 08:00:00,2.0,52.0,1.669073,2.962127
9,7.1,83.9,2000-01-01 09:00:00,4.0,53.1,0.754475,7.059799


A visual inspection of the dataframe above indicates that the wind vectorizing function is behaving as desired. Incorporate it into a `ColumnTransformer`:

In [12]:
wind_vect_trans = ColumnTransformer(transformers = [
    ("wind-vectorizer", FunctionTransformer(vectorize), ["WS", "WD"]),
    ], remainder = "passthrough")

## Preprocessing setup: Prepare to scale the data.

As the last step of preprocessing, the data will be Z-scaled. Any remaining datetime data should be dropped here.

In [13]:
# For reasons that are not clear, the scaled value must explicitly be cast as float64 type.
# Otherwise, the pipeline that calls this transformer returns the values as a string rather than float.

def zscaler(df):
    #for feature in df.select_dtypes("number").columns:  # This yields an otherwise working transformer that scales the data and returns str rather than float
    for feature in df.columns:                         # This yields an otherwise working transformer that z-scales the datetime
        df[feature] = ((df[feature] - df[feature].mean()) / df[feature].std()).astype("float64")
    return df

In [14]:
def ozone_copier(df):
    df["O3-untrans"] = df["O3"]
    return df

In [15]:
scaler = ColumnTransformer(transformers = [
    ("standard-scaler", FunctionTransformer(zscaler), ["N_wind_vec", "E_wind_vec", "T", "O3"]),
    ], remainder = "passthrough")

## Preprocessing setup: Assemble the completed transformer

Assemble the preprocessing steps from prior sections (transforming, restructuring tabular data) into a preprocessing transformer that also includes Z-scaling.

In [16]:
# For convenience, the partially processed array will be re-transformed back in to a dataframe partway through
# using the `dataframer` FunctionTransformer so that the scaling transformer can use pandas-derived functions.
dataframer = FunctionTransformer(lambda x: pd.DataFrame(x, columns = ["N_wind_vec", "E_wind_vec", "DateTime", "O3", "T"])) 

In [17]:
# After completing preprocessing, convert the data back to a dataframe to allow straightforward restructuring using the structure_timeseries function.
dataframer2 = FunctionTransformer(lambda x: pd.DataFrame(x, columns = ["N_wind_vec", "E_wind_vec", "T", "O3", "DateTime", "O3-untrans"]).convert_dtypes()) 

In [18]:
preprocessor = Pipeline(steps = [
    ("wind-processor", wind_vect_trans),
    ("re-dataframer1", dataframer),
    ("o3-copier", FunctionTransformer(ozone_copier)),
    ("z-scaler", scaler),
    ("re-dataframer2", dataframer2),
    ])

Visually spot-check to ensure that the preprocessing pipeline functions as expected.

In [19]:
transformed = preprocessor.fit_transform(df)
transformed

,N_wind_vec,E_wind_vec,T,O3,DateTime,O3-untrans
0,0.295402,0.307083,-1.056546,-1.067699,2000-01-01 00:00:00,1.0
1,0.426579,0.318703,-1.047041,-1.067699,2000-01-01 01:00:00,1.0
2,0.391015,0.378467,-1.02803,-1.067699,2000-01-01 02:00:00,1.0
3,0.421437,0.486946,-1.047041,-1.013463,2000-01-01 03:00:00,2.0
4,0.566795,0.544784,-1.066051,-1.121936,2000-01-01 04:00:00,0.0
...,...,...,...,...,...,...
157132,0.652639,0.405979,-1.199126,-0.850753,2021-12-31 19:00:00,5.0
157133,0.923627,0.340336,-1.199126,-0.796517,2021-12-31 20:00:00,6.0
157134,0.658357,0.373295,-1.294179,-0.90499,2021-12-31 21:00:00,4.0
157135,0.919802,0.389852,-1.484286,-1.067699,2021-12-31 22:00:00,1.0


In [20]:
transformed.describe()

,N_wind_vec,E_wind_vec,T,O3,O3-untrans
count,1.571370e+05,1.571370e+05,1.571370e+05,1.571370e+05,157137.000000
mean,4.603558e-15,3.407993e-15,-2.019908e-14,-8.084986e-16,20.686017
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,18.437794
min,-1.963407e+01,-1.026788e+01,-3.603974e+00,-1.121936e+00,0.000000
25%,-7.235632e-01,-6.740653e-01,-7.143544e-01,-9.592263e-01,3.000000
50%,-4.927817e-02,1.766493e-01,-9.650811e-02,-1.999164e-01,17.000000
75%,7.277915e-01,7.003873e-01,6.068861e-01,7.221028e-01,34.000000
max,1.464476e+01,1.183780e+01,4.608629e+00,7.176237e+00,153.000000


In [21]:
restruct_dt, restruct_y, restruct_x = structure_timeseries(transformed, dt_col = "DateTime", y_col = "O3-untrans", window = 24)

Visually inspect the transformed & restructured data:

In [22]:
restruct_dt[:5]

[Timestamp('2000-01-02 01:00:00'),
 Timestamp('2000-01-02 02:00:00'),
 Timestamp('2000-01-02 03:00:00'),
 Timestamp('2000-01-02 04:00:00'),
 Timestamp('2000-01-02 05:00:00')]

In [23]:
restruct_y[:5]

[1.0, 4.0, 6.0, 0.0, 33.0]

## Split the data into training & test sets

In [24]:
restruct_x[:1]

[array([[0.5039213837916564, 0.3427103484422527, -1.3797272805914906,
         -1.0134627009662223, 2.0],
        [0.6772141798281631, 0.4156139558624601, -1.3607166262059487,
         -1.0134627009662223, 2.0],
        [0.4135705090926748, 0.3930586727877959, -1.3702219533987194,
         -1.0134627009662223, 2.0],
        [0.3881321618399058, 0.6492088772653191, -1.3702219533987194,
         -1.0134627009662223, 2.0],
        [0.5766723097493203, 0.38687927093546876, -1.2846740086637791,
         -1.0676991230762298, 1.0],
        [-0.37976316392754667, 0.3831372084414142, -1.2086313911216102,
         -0.9592262788562149, 3.0],
        [-0.586211919657543, -0.15250458434334732, -1.1801154095432964,
         -0.63380774619617, 9.0],
        [-0.7011904091854805, -0.25114216109624904, -1.0660514832300432,
         -0.0914435250960952, 19.0],
        [-1.004205496887869, 0.4480468203791252, -0.9519875569167893,
         -0.0914435250960952, 19.0],
        [-0.9510001040320317, 0.450635

The data has been successfully transformed & restructured.

### Split the data set

Use the first 75% of the dataset for training, with the balance held out for testing.

In [25]:
train_frac = 0.75
slice_pt = int(len(restruct_dt)*train_frac)

In [26]:
x_train, x_test = restruct_x[:slice_pt], restruct_x[slice_pt:]
len(restruct_x), len(x_train), len(x_test)

(157112, 117834, 39278)

In [27]:
y_train, y_test = restruct_y[:slice_pt], restruct_y[slice_pt:]
len(restruct_y), len(y_train), len(y_test)

(157112, 117834, 39278)

In [28]:
dt_train, dt_test = restruct_dt[:slice_pt], restruct_x[slice_pt:]
len(restruct_dt), len(dt_train), len(dt_test)

(157112, 117834, 39278)

For ease of subsequent saving / loading, convert each to a numpy array:

In [29]:
x_train, x_test = (np.array(x_train),
                   np.array(x_test))
x_train.shape

(117834, 24, 5)

In [30]:
y_train, y_test = (np.array(y_train),
                   np.array(y_test))
y_train.shape

(117834,)

In [31]:
dt_train, dt_test = (np.array(dt_train),
                   np.array(dt_test))
dt_train.shape

(117834,)

## Save everything

Export processed data.

In [32]:
np.savez_compressed("O3_model/transformed_datasets.npz",
                    x_train = x_train,
                    x_test = x_test,
                    y_train = y_train,
                    y_test = y_test,
                    dt_train = dt_train,
                    dt_test = dt_test)

Ensure fidelity of the saved arrays:

In [33]:
datasets = np.load("O3_model/transformed_datasets.npz",
                   allow_pickle = True)

In [34]:
for array_name in datasets.files:
    print(array_name), print(type(array_name))

x_train
<class 'str'>
x_test
<class 'str'>
y_train
<class 'str'>
y_test
<class 'str'>
dt_train
<class 'str'>
dt_test
<class 'str'>


In [35]:
loaded_datasets = {}

for array_name in datasets.files:
    loaded_datasets[array_name] = datasets[array_name]
    
    if not np.array_equal(loaded_datasets[array_name], eval(array_name)):
        print("Problem with " + array_name)                                             

Also save the fitted preprocessing pipeline.

In [36]:
from dill import dump, load

In [37]:
with open("O3_model/preprocessor_pipeline.dill", "wb") as file:
    dump(preprocessor, file)

Again ensure that this was saved correctly:

In [38]:
with open("O3_model/preprocessor_pipeline.dill", "rb") as file:
    preprocessor2 = load(file)

In [39]:
preprocessor.transform(df).equals(preprocessor2.transform(df))

True